## Connecting to Snowflake Snowpark and setting up the initial session

In [ ]:
from typing import Dict, Union
from snowflake.snowpark import Session

connection_parameters: Dict[str, Union[int, str]] = {
    "account": "<account-identifier>",
    "user": "<username>",
    "password": "<password>",
    "role": "ACCOUNTADMIN",
    "database": "snowpark_test_db",
    "schema": "snowpark_test_schema"
}

In [ ]:
session = Session.builder.configs(connection_parameters).create()

## Testing a simple SELECT statement 

In [ ]:
df = session.sql('SELECT * FROM snowflake_sample_data.tpch_sf10.orders')

df.show()

In [ ]:
print(session.get_current_account())
print(session.get_current_database())
print(session.get_current_role())
print(session.get_current_schema())

## Creating a dataframe and querying the resulting table

In [ ]:
session.sql('CREATE OR REPLACE TABLE sample_product_data (id INT, parent_id INT, category_id INT, name VARCHAR, serial_number VARCHAR, key INT, "3rd" INT)').collect()

session.sql("""
             INSERT INTO sample_product_data VALUES
            (1, 0, 5, 'Product 1', 'prod-1', 1, 10),
            (2, 1, 5, 'Product 1A', 'prod-1-A', 1, 20),
            (3, 1, 5, 'Product 1B', 'prod-1-B', 1, 30),
            (4, 0, 10, 'Product 2', 'prod-2', 2, 40),
            (5, 4, 10, 'Product 2A', 'prod-2-A', 2, 50),
            (6, 4, 10, 'Product 2B', 'prod-2-B', 2, 60),
            (7, 0, 20, 'Product 3', 'prod-3', 3, 70),
            (8, 7, 20, 'Product 3A', 'prod-3-A', 3, 80),
            (9, 7, 20, 'Product 3B', 'prod-3-B', 3, 90),
            (10, 0, 50, 'Product 4', 'prod-4', 4, 100),
            (11, 10, 50, 'Product 4A', 'prod-4-A', 4, 100),
            (12, 10, 50, 'Product 4B', 'prod-4-B', 4, 100)
            """).collect()

In [ ]:
session.sql("SELECT count(*) FROM sample_product_data").collect()

In [ ]:
df_table = session.table("sample_product_data")

In [ ]:
df_table.show()

In [ ]:
df1 = session.create_dataframe([1, 2, 3, 4]).to_df("a")
df1.show()

In [ ]:
df2 = session.create_dataframe([[1, 2, 3, 4]], schema=["a", "b", "c", "d"])
df2.show()

## Creating simple UDFs

In [ ]:
import numpy as np
import pandas as pd

from snowflake.snowpark.functions import udf

session.add_packages("numpy", "pandas")

# This cell was run both without the name="compute" argument and with. Without this name, Snowpark will store multiple versions of the function with unique names. With the argument, this cell will fail to execute on account of the 'COMPUTE' object already existing.
@udf(name="compute")
def compute() -> list:
    return [np.__version__, pd.__version__]

In [ ]:
result = session.sql("SELECT FUNCTION_NAME FROM SNOWPARK_TEST_DB.INFORMATION_SCHEMA.FUNCTIONS WHERE FUNCTION_SCHEMA = 'SNOWPARK_TEST_SCHEMA'").collect()

for row in result:
    print(row[0])

In [ ]:
result = session.sql("SELECT SNOWPARK_TEST_SCHEMA.SNOWPARK_TEMP_FUNCTION_I8NPUJLPJ3()").collect()

raw_output = result[0][0]

data_list = eval(raw_output) # type: ignore

# print(result[0])
numpy_version, pandas_version = data_list

print(numpy_version, pandas_version)

In [ ]:
result = session.sql("SELECT SNOWPARK_TEST_SCHEMA.compute()").collect()

raw_output = result[0][0]

data_list = eval(raw_output) # type: ignore

# print(result[0])
numpy_version, pandas_version = data_list

print(numpy_version, pandas_version)

### Anonymous UDF

In [ ]:
from snowflake.snowpark.types import IntegerType
from snowflake.snowpark.functions import udf

add_one = udf(lambda x: x+1, return_type=IntegerType(), input_types=[IntegerType()])

In [ ]:
result = session.sql("SELECT SNOWPARK_TEST_SCHEMA.SNOWPARK_TEMP_FUNCTION_5OGDJ5Y3I3(1)").collect()

print(result[0][0])

## Dropping UDFs

In [ ]:
result = session.sql("SELECT FUNCTION_NAME FROM SNOWPARK_TEST_DB.INFORMATION_SCHEMA.FUNCTIONS WHERE FUNCTION_SCHEMA = 'SNOWPARK_TEST_SCHEMA'").collect()

for row in result:
    print(row[0])

In [ ]:
session.sql("DROP FUNCTION if exists SNOWPARK_TEST_SCHEMA.SNOWPARK_TEMP_FUNCTION_Y2XQ4S2YP6()").collect()

In [ ]:
result = session.sql("SELECT FUNCTION_NAME FROM SNOWPARK_TEST_DB.INFORMATION_SCHEMA.FUNCTIONS WHERE FUNCTION_SCHEMA = 'SNOWPARK_TEST_SCHEMA'").collect()

for row in result:
    print(row[0])

## Closing the session

In [ ]:
session.close()